## functionalities

1. build a backing track, specify functional harmony, then generate improvisation.
* notes
* rhythm

2. given a backing track defined, do reharmonization.

3. given a chord and a corresponding chord name, figure out the closest chord in terms of voicing.

In [1]:
from midiutil import MIDIFile
import numpy as np
import pygame
import pandas as pd
from collections import deque 
from functools import reduce


pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


## what is a note:

note has properties of:
* name: A B C D 
* shift and n: actual numbers
* spectrum



## what is a chord/mode

it's a lot more complicated. Let's start with use cases.

When formulating a song, the input would always be using modes first. i.e.:

* We're in C Ionian key, the chord progression is IV, III, II, I.
* Sometimes we can add secondary dominance. F Ionian(II V)C Ionian(IV III II I).
* Sometimes we can even add more passing chords that points to no modes. It's just a brush of color.
* Even when the chords are in the mode, we can do alternation. For example V alt dominant chord.

So there are two ways to set up chord progressions:

1. functional harmony framework, with the freedom of altered notes
2. non-functional chords



1. modeling basic music components.
2. design input and output and add-ins. 
3. Variations, pattern recognition

* Input would be formulating a song. 
* Output would be midi files as well as a dataframe to represent the song and solos.
* add-ins: customized patterns for backing tracks.
* Pattern

voicing module is for customized chord voicings and


A solo module.

In [2]:
class Note:
    """
    name is the note of a chord.
    n represent relative location in the twelve equal temperament
    shift = 0 means 4th octave.
    """
    def __init__(self,name=None,n=None,shift=0,*args, **kwargs):
        note_name_l = ['C','C#','Db','D','D#',\
                              'Eb','E','F','F#','Gb','G','G#','Ab','A','A#','Bb','B']
        note_loc_l = [0, 1, 1, 2, 3, 3, 4, 5, 6, 6, 7, 8, 8, 9, 10, 10, 11]
        convert_dict = dict(zip(note_name_l,note_loc_l))
        rev_convert_dict = dict(zip(note_loc_l,note_name_l))
        if name and (n is None):
            self.name = name
            self.shift = shift
            self.n = convert_dict[self.name]+(self.shift+5)*12
        elif (n is not None) and (name is None):
            self.n = n%12
            if shift is None:
                self.shift = n//12-5
            else:
                self.shift = shift
            self.name = rev_convert_dict[self.n]
            
        self.repr_str = f"Note: {self.name}"
        self.spectrum = [self.n+k for k in [12*(i-3) for i in range(7)]]
            
    def __repr__(self):
        return self.repr_str 
    
    def __str__(self):
        return self.repr_str 
    
    def __sub__(self,other_note):
        """
        measure the distance of two notes. Important right?
        
        other_note is always the chord notes (or extensions); self note is the root.
        """
        
        return (other_note.n%12-self.n)%12
    
    def __members(self):
        return (self.name)
    
    def __eq__(self, other):
        if type(other) is type(self):
            return self.__members() == other.__members()
        else:
            return False

    def __hash__(self):
        return hash(self.__members())


In [8]:
print(Note("C").n, Note(n=71),Note("D")-Note("C"))

60 Note: B 10


In [16]:
class Mode:
    def __init__(self,key,mode_name=None,*args, **kwargs):
        self.key = key
        self.mode_name = mode_name

        
        mode_name_l = ['Ionian','Dorian','Phrygian','Lydian','Mixolydian','Aeolian','Locrian']
        sequence_l = [0,1,2,3,4,5,6]
        name_dict = dict(zip(mode_name_l,sequence_l))
        
        self.sequence = name_dict[self.mode_name]
        
        self.key_note = Note(self.key).n
        
        circle_of_fifth_distance = [2,2,1,2,2,2,1]
        
        mode_distance = deque(circle_of_fifth_distance)
        mode_distance.rotate(-1*self.sequence)
        
        self.current_spin = mode_distance
        mode_distance = [0]+list(self.current_spin)[:-1]

        self.note_n_l = np.cumsum(mode_distance)+self.key_note
        self.note_l = [Note(n=i) for i in self.note_n_l]
        self.repr_str = f"Key: {self.key} - {self.mode_name}\nNotes: {','.join([i.name for i in self.note_l])}"
        
        self.scale = dict(zip(range(1,8),self.note_l))

    def __repr__(self):
        return self.repr_str
    
    def __str__(self):
        return self.repr_str

In [35]:
class Chord:
    """
    degree: 1,2,3,4,5,6,7
    1. specify root
    2. specify one of: quality, mode_name, other_notes
    or mode_name and degree (of the bar)
    """
    def __init__(self,root=None,quality=None, other_notes = None,extensions = None, *args, **kwargs):

        self.root = root
        self.root_note = Note(self.root)
        self.mode = None
        self.extension_input = extensions
        self.extension = None
        
        mode_quality_l= ['Maj7','m7','7','m7b5']
        mode_list = ['Ionian','Dorian','Mixolydian','Locrian']
        mode_distance_list = [frozenset([4,7,11]),frozenset([3,7,10]),frozenset([4,7,10]),frozenset([3,6,10])]
        
        quality_list = mode_quality_l+['dim']
        distance_list = mode_distance_list+[frozenset([3,6,9])]
        
        quality2mode_d = dict(zip(mode_quality_l,mode_list))
        
        distance2quality_d = dict(zip(distance_list,quality_list))
        quality2distance_d = dict(zip(quality_list,distance_list))

        if quality is not None:
            self.quality = quality
            if quality in mode_quality_l:
                self.mode_name = quality2mode_d[self.quality]
                self.mode = Mode(key=self.root,mode_name=self.mode_name)
            else:
                self.mode_name = 'no_mode'

                
        elif other_notes:
            distance_set = frozenset([self.root_note - other_note for other_note in other_notes])
            if distance_set in distance_list:
                self.quality = distance2quality_d[distance_set]
                if distance_set in quality2mode_d.keys():
                    self.mode_name = quality2mode_d[self.quality]
                    self.mode = Mode(key=self.root,mode_name=self.mode_name)
            else:
                raise NotImplementedError
        
        if self.mode:
            self.scale = self.mode.scale
            self.chord7 = [self.scale[i] for i in [1,3,5,7]]
        else:
            self.chord7 = [Note(n = self.root_note.n+i) for i in [0]+list(quality2distance_d[self.quality]) ]
        
        if self.extension_input:
            if max(self.extension_input)>=14:
                raise ValueError
            self.extension_input = [i-7 for i in self.extension_input]
            self.extension = [self.scale[i] for i in self.extension_input]
        
        self.name = f'{self.root}{self.quality}'
        
        if self.extension is None:
            self.notes = self.chord7
        else:
            self.notes = self.chord7+self.extension
    def __repr__(self):
        return self.name

    def __str__(self):
        return self.name

In [36]:
print(Chord("C","Maj7"),Chord("C","Maj7",extensions = [9]).notes)

Key: C - Ionian
Notes: C,D,E,F,G,A,B C None
CMaj7 [Note: C, Note: E, Note: G, Note: B, Note: D]


In [37]:
class FunctionalChord(Mode):
    """
    given a mode and its degree, get the right chord under the framework of functional harmony.
    """
    def __init__(self,key=None,mode_name=None,degree=1,extensions = None,*args, **kwargs):
        
        super().__init__(key, mode_name)
        self.degree = degree
        
        roman_n_dict = dict(zip(range(1,8),["I","II","III","IV","V","VI","VII"]))
        
        self.roman_numeral = roman_n_dict[self.degree]
        
        note_deque = deque(self.note_l)
        note_deque.rotate(-1*(self.degree-1))
        
        note_l = list(note_deque)
        
        self.note_l = [note_l[i-1] for i in [1,3,5,7]]
        
        self.chord = Chord(root=self.note_l[0].name,other_notes=self.note_l[1:],extensions = extensions)
        self.repr_str = self.roman_numeral+self.chord.quality
        
    def __repr__(self):
        return self.repr_str
    
    def __str__(self):
        return self.repr_str  

In [38]:
class Improvise:
    def __init__(self,chord_progression,grid_unit=1/16):
        self.cp_df = chord_progression
        self.cp_df['length']
        

In [39]:
class Notes_Playable(Chord):
    """
    given key,mode and the chord name, if it's 
    """
    def __init__(self,key,mode_name,chord_root,chord_quality,other_notes=None,*args, **kwargs):
        super().__init__(root=chord_root,quality=chord_quality, other_notes = other_notes)


        self.key_mode = Mode(key = key,mode_name = mode_name)
        self.key_mode_tone = self.key_mode.note_l
        
        self.chord_tone = [Note(n = i.n,shift = 0) for i in self.chord7]
        self.melody_spectrum = set(self.chord7).union(self.key_mode_tone)
    
    

                

In [40]:
m = FunctionalChord(key = "F", mode_name = 'Ionian',degree = 2)
m.chord

Gm7

In [32]:
k = Notes_Playable(key = 'C',mode_name = 'Ionian',chord_root=m.chord.root,chord_quality = m.chord.quality)

In [33]:
k.chord_tone,k.key_mode_tone

([Note: G, Note: Bb, Note: D, Note: F],
 [Note: C, Note: D, Note: E, Note: F, Note: G, Note: A, Note: B])

In [41]:
chord_progression_df = pd.DataFrame({"bar":[1,2,3,4],"key":['C']*4,"mode":['Ionian']*4,"degree":[2,5,1,1],"bar_time":[1]*4,'extensions':[None,[13],None,None]})
#chord_progression_df = pd.DataFrame({"bar":[1,2,3,4],"key":['C']*4,"mode":['Ionian',"Aeolian","Ionian","Ionian"],"degree":[4,4,3,6],"bar_time":[1]*4})

chord_progression_df['function_chord'] = chord_progression_df.apply(lambda x: (FunctionalChord(key = x.key,mode_name = x['mode'],degree = x.degree,extensions = x.extensions)), axis = 1)
chord_progression_df['chord'] = chord_progression_df.function_chord.apply(lambda x:x.chord)


None G None


AttributeError: 'Chord' object has no attribute 'scale'

In [ ]:
chord_progression_df

In [ ]:
class Composer:
    """
    chord_progression, a chord progression dataframe
    """
    def __init__(self, cp_df, grid_unit = 1/8):
        
        cp_df['unit']=cp_df.bar_time.apply(lambda x: range(1,int(x/grid_unit)+1))  
        self.cp_df = cp_df.explode("unit")
        
    def get_playable_notes(tmp_chord):
        pass
        


In [ ]:
Comp1 = Composer(chord_progression_df)

Comp1.cp_df